In [26]:
cd '/home/lenny/PROJECTS/fastapi-nosql-scrape'


/home/lenny/PROJECTS/fastapi-nosql-scrape


In [27]:
!pip install "lxml[html_clean]"
!pip install lxml_html_clean

You should consider upgrading via the '/home/lenny/PROJECTS/fastapi-nosql-scrape/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/lenny/PROJECTS/fastapi-nosql-scrape/venv/bin/python3 -m pip install --upgrade pip' command.


In [59]:
import requests
from requests_html import HTML
from app import scraper
from slugify import slugify
import pprint
import re

In [60]:
url = "https://www.amazon.com/Vitamix-E310-Explorian-Professional-Grade-Container/dp/B0758JHZM3/ref=sr_1_3?dchild=1&keywords=vitamix&qid=1632337952&sr=8-3&th=1"

s = scraper.Scraper(url=url, endless_scroll=True)

In [61]:
html_str = s.get()

In [62]:
html_obj = HTML(html=html_str)

In [63]:
tables = html_obj.find("table")
tables

[<Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-normal', 'a-spacing-none', 'reInventPriceTable')>,
 <Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-normal', 'a-spacing-micro')>,
 <Element 'table' class=('a-normal',)>,
 <Element 'table' class=('a-normal',)>,
 <Element 'table' class=('a-normal',)>,
 <Element 'table' class=('a-normal',)>,
 <Element 'table' class=('a-normal',)>,
 <Element 'table' id='productDetails_detailBullets_sections1' class=('a-keyvalue', 'prodDetTable')>,
 <Element 'table' id='productDetails_feedback_sections' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' class=('a-bordered', 'a-horizontal-stripes', 'a-spacing-none', 'a-size-small', '_product-comparison-desktop_desktopFaceoutStyle_comparisonTable__hYFf4') data-mix-operations='process-swatch-click'>]

In [64]:
title_str = html_obj.find("#productTitle", first=True).text
title_str

'Vitamix E310 Explorian Blender, Professional-Grade, 48 Oz. Container, Black'

In [65]:
def extract_element_text(html_obj, element_id):
    el =  html_obj.find(element_id, first=True)
    if not el:
        return ''
    return el.text

price_str = extract_element_text(html_obj, '#priceblock_ourprice')
price_str

''

In [66]:
def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    val = None
    if len(x) == 1:
        val = x[0]
    return val

In [67]:


def extract_dataset(tables):
    dataset = {}
    for table in tables:
        for tbody in table.element.getchildren():
            for tr in tbody.getchildren():
                row = []
                for col in tr.getchildren():
                    content = ""
                    try:
                        content = col.text_content()
                    except:
                        pass
                    if content != "":
                        _content = content.strip()
                        row.append(_content)
                if len(row) != 2:
                    continue
                key = row[0]
                value = row[1]
                
                # print(key, value)
                data = {}
                key = slugify(key)
                if key in dataset:
                    continue
                else:
                    if "$" in value:
                        new_key = key
                        old_key = f'{key}_raw'
                        new_value = extract_price_from_string(value)
                        old_value = value
                        dataset[new_key] = new_value
                        dataset[old_key] = old_value
                    else:
                        dataset[key] = value
    return dataset



In [68]:
dataset = extract_dataset(tables)
pprint.pprint(dataset)

{'': 'Material BPA-free Eastman Tritan® copolyester',
 'asin': 'B0758JHZM3',
 'assembly-required': 'No',
 'batteries-required': 'No',
 'best-sellers-rank': '#2,061 in Kitchen & Dining (See Top 100 in Kitchen & '
                      'Dining)  #12 in Countertop Blenders',
 'blade-material': 'Stainless Steel',
 'brand': 'Vitamix',
 'capacity': '48 Fluid Ounces',
 'care-instructions': 'Dishwasher Safe',
 'color': 'Black',
 'controls-type': 'Physical',
 'country-of-origin': 'USA',
 'customer-reviews': '4.6  4.6 out of 5 stars    \n'
                     '    5,288 ratings   \n'
                     '                    \n'
                     '                    var '
                     'dpAcrHasRegisteredArcLinkClickAction;\n'
                     "                    P.when('A', "
                     "'ready').execute(function(A) {\n"
                     '                        if '
                     '(dpAcrHasRegisteredArcLinkClickAction !== true) {\n'
                     ' 